https://stepik.org/lesson/309343/step/5

In [ ]:
#@title Выполнение свёртки вручную через циклы
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super().__init__(in_channels, out_channels, kernel_size, stride)
        self.padding = 0  # Для простоты padding фиксирован

    def set_kernel(self, kernel):
        self.kernel = kernel

    def __call__(self, input_tensor):
        batch_size, _, input_height, input_width = input_tensor.shape
        output_shape = calc_out_shape(input_tensor.shape, self.out_channels,
                                      self.kernel_size, self.stride, self.padding)
        output_tensor = torch.zeros(output_shape)  # Создание выходного тензора

        # Выполнение свёртки вручную через циклы
        for b in range(batch_size):  # Перебор по batch
            for oc in range(self.out_channels):  # Перебор по каналам выхода
                for oy in range(output_shape[2]):  # Перебор по высоте выходного тензора
                    for ox in range(output_shape[3]):  # Перебор по ширине выходного тензора
                        for ic in range(self.in_channels):  # Перебор по каналам входа
                            for ky in range(self.kernel_size):  # Перебор по высоте ядра
                                for kx in range(self.kernel_size):  # Перебор по ширине ядра
                                    # iy и ix — это координаты соответствующего элемента во входном тензоре
                                    # oy и ox — текущие координаты элемента в выходном тензоре
                                    # self.stride — шаг свёртки
                                    # ky и kx — это координаты элемента внутри ядра свёртки
                                    iy = oy * self.stride + ky  # формула учитывает шаг (self.stride),
                                    ix = ox * self.stride + kx  # чтобы правильно вычислить позицию на входе№
                                    # Проверка выхода за границы:
                                    # проверка гарантирует, что вычисленные координаты iy и ix находятся внутри границ входного тензора
                                    if 0 <= iy < input_height and 0 <= ix < input_width:
                                        output_tensor[b, oc, oy, ox] += (
                                            input_tensor[b, ic, iy, ix] * self.kernel[oc, ic, ky, kx]
                                        )

        return output_tensor

# Проверка корректности реализации
print(test_conv2d_layer(Conv2dLoop))


True


In [ ]:
#@title Вспомогательный код
import torch
from abc import ABC, abstractmethod


# абстрактный класс для сверточного слоя
class ABCConv2d(ABC):
    # инициализирует основные параметры свёртки
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        # Количество каналов на входе свёрточного слоя. Для (RGB) будет = 3
        self.in_channels = in_channels
        # Количество каналов на выходе свёрточного слоя. Для (RGB) будет = 3
        self.out_channels = out_channels
        # Размер ядра свёртки. если = 3, то ядро свёртки будет размером 3x3
        self.kernel_size = kernel_size
        # Шаг свёртки
        self.stride = stride

    # Метод позволяет задавать весовые коэффициенты (ядро свёртки).
    def set_kernel(self, kernel):
        self.kernel = kernel
    '''
    Метод __call__ в Python — это специальный метод, который позволяет экземплярам класса вести себя как функции.
    Если класс определяет метод __call__, вы можете вызывать объекты этого класса так же, как и обычные функции.
    '''
    # В нашем абстрактном классе ABCConv2d метод __call__ объявлен как абстрактный метод:
    @abstractmethod
    # Значит любые подклассы, наследующиеся от ABCConv2d, обязаны реализовать этот метод.
    # В конкретных реализациях, таких как класс Conv2d, метод __call__ будет выполнять свёртку над входным тензором.
    def __call__(self, input_tensor):
        pass


# класс-обертка над torch.nn.Conv2d для унификации интерфейса
# класс реализует конкретный свёрточный слой, используя torch.nn.Conv2d из PyTorch
class Conv2d(ABCConv2d):
    # Конструктор создаёт объект torch.nn.Conv2d, который будет выполнять свёртку
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)
    # задаёт веса (ядро) свёртки.
    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        # реализует свёртку, вызывая свёрточный слой на входном тензоре
        return self.conv2d(input_tensor)


# функция:
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    # извлекает нужные размеры из ядра
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]
    # создаёт экземпляр класса свёрточного слоя conv2d_layer_class
    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    # Устанавливает ядро свёртки (методом из астрактного класса)
    layer.set_kernel(kernel)
    # Выполняет свёртку над входным тензором input_matrix и возвращает результат
    return layer(input_matrix)


# Функция, тестирующая класс conv2d_cls.
# Возвращает True, если свертка совпадает со сверткой с помощью torch.nn.Conv2d.
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    # Создаёт ядро свёртки
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    # Подсчитывает количество каналов в ядре
    in_channels = kernel.shape[1]

    # Создаёт входной тензор
    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    # Вызывает функцию для создания и выполнения свёртки
    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    # Сравнивает результат с эталонной реализацией на основе Conv2d
    # Возвращает True, если результаты и размеры совпадают
    return torch.allclose(custom_conv2d_out, conv2d_out) \
        and (custom_conv2d_out.shape == conv2d_out.shape)

# Вызов запускает тест и выводит True, если свёртка, выполненная классом Conv2d, работает корректно
print(test_conv2d_layer(Conv2d))

True


In [ ]:
#@title Вспомогательный код
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    # Конструктор создаёт объект torch.nn.Conv2dLoop, который будет выполнять свёртку
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)
    # задаёт веса (ядро) свёртки.
    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        # реализует свёртку, вызывая свёрточный слой на входном тензоре
        return self.conv2d(input_tensor)

# Корректность реализации определится в сравнении со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dLoop))

True
